In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from typing import TypedDict, Literal, Annotated, List
from operator import add
from langchain_core.messages import HumanMessage, SystemMessage, BaseMessage
from langgraph.graph import END, START, StateGraph
from langgraph.prebuilt import ToolNode
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool


class InputState(TypedDict):
    article: str

class OutputState(TypedDict):
    agent_output: str

class OverallState(InputState, OutputState):
    messages: Annotated[List[BaseMessage], add]

# 도구 정의: 국가별 GDP 조회
@tool
def get_gdp(country_name: str) -> str:
    """Fetch the latest GDP for a given country (simulated)."""
    fake_gdp_data = {
        "United States": "26.7 trillion USD",
        "South Korea": "1.8 trillion USD",
        "Japan": "4.2 trillion USD",
    }
    return fake_gdp_data.get(country_name, "GDP data not available.")

# 🔗 도구 바인딩
tools = [get_gdp]
model = ChatOpenAI(model="gpt-4o-mini").bind_tools(tools)

# 🧠 LLM 호출 노드
def call_model_gdp_info(state: OverallState):
    local_messages = state.get("messages", [])
    if not local_messages:
        human_message = HumanMessage(content=state["article"])
        local_messages.append(human_message)

    system_message = SystemMessage(
        content="""You are an economic research agent tasked with identifying the latest GDP of a country.
                If GDP data is missing, call the appropriate tool to retrieve it. 
                Provide a clear and concise summary."""
    )

    response = model.invoke([system_message] + local_messages)
    state["agent_output"] = response.content
    state["messages"] = local_messages + [response]
    return state

# 🔁 조건 분기 (tool 호출 여부 판단)
def should_continue(state: OverallState) -> Literal["tools", END]:
    last_message = state["messages"][-1]
    if getattr(last_message, "tool_calls", None):
        return "tools"
    return END

# 🧭 LangGraph 구성
gdp_graph = StateGraph(OverallState, input=InputState, output=OutputState)
gdp_graph.add_node("call_model_gdp_info", call_model_gdp_info)
gdp_graph.add_node("tools", ToolNode(tools))
gdp_graph.add_edge(START, "call_model_gdp_info")
gdp_graph.add_conditional_edges("call_model_gdp_info", should_continue)
gdp_graph.add_edge("tools", "call_model_gdp_info")

gdp_research_agent = gdp_graph.compile()


In [3]:
initial_state = {
    "article": "대한민국의 현재 GDP에 대해 알려줘.",
}

result = gdp_research_agent.invoke(initial_state)

print("Agent Output:\n", result["agent_output"])

Agent Output:
 대한민국의 현재 GDP는 약 1.8조 달러입니다.


In [ ]:
initial_state = {
    "article": "대만의 현재 GDP에 대해 알려줘.",
}

gdp_research_agent.invoke(initial_state)

{'agent_output': '현재 대만의 GDP 데이터는 이용할 수 없습니다. 추가적인 정보가 필요하다면 다른 질문을 해주세요.'}

In [5]:
from typing import TypedDict, Literal, Annotated, List
from operator import add
from langchain_core.messages import HumanMessage, SystemMessage, BaseMessage
from langgraph.graph import END, START, StateGraph
from langgraph.prebuilt import ToolNode
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool


class InputState(TypedDict):
    article: str

class OutputState(TypedDict):
    agent_output: str

class OverallState(InputState, OutputState):
    messages: Annotated[List[BaseMessage], add]


# 국가 환율 조회 도구
@tool
def get_exchange_rate(country_name: str):
    """Gets the current exchange rate of a country (to USD)"""
    fake_exchange_db = {
        "South Korea": "1 USD = 1,350 KRW",
        "Japan": "1 USD = 150 JPY",
        "Eurozone": "1 USD = 0.92 EUR",
    }
    return fake_exchange_db.get(country_name, "Exchange rate information not available.")

# 도구 바인딩
tools2 = [get_exchange_rate]
model2 = ChatOpenAI(model="gpt-4o-mini").bind_tools(tools2)

# LLM 노드 정의
def call_model_exchange_rate(state: OverallState):
    local_messages = state.get("messages", [])
    if not local_messages:
        human_message = HumanMessage(content=state["article"])
        local_messages.append(human_message)
    system_message = SystemMessage(
        content="""You are an economic agent tasked with finding the current exchange rate of a given country.
                    If the rate is unavailable, use the appropriate tool to fetch the data."""
    )
    response = model2.invoke([system_message] + local_messages)
    state["agent_output"] = response.content
    state["messages"] = local_messages + [response]
    return state

# 조건 분기
def should_continue(state: OverallState) -> Literal["tools", END]:
    last_message = state["messages"][-1]
    if getattr(last_message, "tool_calls", None):
        return "tools"
    return END

# 그래프 구성
exchange_graph = StateGraph(OverallState, input=InputState, output=OutputState)
exchange_graph.add_node("call_model_exchange_rate", call_model_exchange_rate)
exchange_graph.add_node("tools", ToolNode(tools2))
exchange_graph.add_edge(START, "call_model_exchange_rate")
exchange_graph.add_conditional_edges("call_model_exchange_rate", should_continue)
exchange_graph.add_edge("tools", "call_model_exchange_rate")

exchange_rate_agent = exchange_graph.compile()

In [6]:
initial_state = {
    "article": "한국과 미국의 교역이 10% 증가할 것 같습니다.",
}

result = exchange_rate_agent.invoke(initial_state)

print("💱 Exchange Rate Info:\n", result["agent_output"])

💱 Exchange Rate Info:
 한국과 미국 간의 교역이 10% 증가하는 것은 두 나라의 경제에 긍정적인 영향을 미칠 수 있습니다. 이러한 증가가 예상되는 이유는 여러 가지가 있을 수 있으며, 예를 들어 무역 마찰 완화, 새로운 무역 협정, 글로벌 경제 회복 등이 있을 수 있습니다.

교역이 증가하면 다음과 같은 효과가 있을 수 있습니다:

1. **경제 성장**: 교역 증가로 인해 양국의 GDP가 증가할 가능성이 있습니다.
2. **고용 창출**: 수출입 활동이 활발해짐에 따라 관련 산업에서의 고용이 증가할 수 있습니다.
3. **소비자 선택 확대**: 다양한 상품과 서비스가 유입되면서 소비자 선택의 폭이 넓어질 것입니다.
4. **국제 관계 강화**: 경제적 관계가 강화됨에 따라 양국 간의 정치적 관계도 긍정적인 영향을 받을 수 있습니다.

이와 관련하여 특정한 수치나 데이터를 원하신다면, 예를 들어 교역 규모나 환율 정보를 제공해 드릴 수 있습니다. 필요하신 정보가 있으면 말씀해 주세요!


In [7]:
initial_state = {
    "article": "대만과 유럽의 교역이 5% 증가할 것 같습니다.",
}

result = exchange_rate_agent.invoke(initial_state)

print("💱 Exchange Rate Info:\n", result["agent_output"])

💱 Exchange Rate Info:
 대만과 유럽 간의 교역이 5% 증가한다는 것은 두 지역 간의 경제적 관계가 강화되고 있다는 긍정적인 신호입니다. 이로 인해 양측은 더 많은 상품과 서비스를 수출입하게 될 것이며, 이는 경제 성장과 일자리 창출에 긍정적인 영향을 미칠 수 있습니다. 또한, 투자 기회나 기술 이전과 같은 추가적인 이익도 기대할 수 있습니다.

특정 문제나 추가적인 정보가 필요하시면 말씀해 주세요!


In [8]:
class InputState(TypedDict):
    article: str


class OutputState(TypedDict):
    agent_output: str


class OverallState(InputState, OutputState):
    pass


model_text_writer = ChatOpenAI(model="gpt-4o-mini")


def expand_text_to_100_words(state: OverallState):
    human_message = HumanMessage(content=state["article"])
    system_message = SystemMessage(
        content="Expand the following text to be at least 100 words. Maintain the original meaning while adding detail."
    )
    response = model_text_writer.invoke([system_message, human_message])
    state["agent_output"] = response.content
    return state


text_writer_graph = StateGraph(OverallState, input=InputState, output=OutputState)
text_writer_graph.add_node("expand_text_to_100_words", expand_text_to_100_words)
text_writer_graph.add_edge(START, "expand_text_to_100_words")
text_writer_graph.add_edge("expand_text_to_100_words", END)

text_writer_agent = text_writer_graph.compile()

In [9]:
text_writer_agent.invoke(
    {"article": "한국과 미국의 교역이 10% 증가할 것 같습니다."}
)

{'agent_output': '현재 한국과 미국 간의 교역이 약 10% 증가할 것으로 예상되고 있습니다. 이는 양국 간의 경제적 유대가 더욱 강화되고 있다는 긍정적인 신호로 해석될 수 있습니다. 한국은 다양한 산업 분야에서 경쟁력을 갖추고 있으며, 미국은 세계 최대의 경제 대국으로서 한국 제품에 대한 수요가 증가하고 있습니다. 이러한 교역 증가의 배경에는 양국 간의 무역 협정 및 경제 협력 강화가 있기도 하며, 서로의 시장 접근성이 개선되고 있기 때문입니다. 이로 인해 한국 기업들은 미국 시장에서의 기회를 확대하고, 미국 기업들은 한국의 혁신적인 제품과 기술을 접할 수 있는 기회를 얻게 될 것입니다. 이러한 상호 이익은 양국 경제 성장에 긍정적인 영향을 미칠 것으로 기대됩니다.'}

# Supervisor Agent

In [18]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

# Supervisor Agent
class ArticlePostabilityGrader(BaseModel):
    off_or_ontopic: str = Field(description="The article is about economics or not, 'yes' or 'no'")
    mentions_gdp_value: str = Field(description="The article mentions GDP or economic value, 'yes' or 'no'")
    mentions_exchange_value: str = Field(description="The article mentions exchange rate or economic context, 'yes' or 'no'")
    meets_100_words: str = Field(description="The article has at least 100 words, 'yes' or 'no'")

llm_postability = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm_postability_grader = llm_postability.with_structured_output(
    ArticlePostabilityGrader
)

postability_system = """
You are a grader assessing whether a news article meets the following criteria:
1. The article is about economics. If yes, answer 'yes'; otherwise, 'no'.
2. The article explicitly mentions GDP or economic value. If yes, answer 'yes'; otherwise, 'no'.
3. The article mentions exchange rate or relevant economic information. If yes, answer 'yes'; otherwise, 'no'.
4. The article contains at least 100 words. If yes, answer 'yes'; otherwise, 'no'.
"""

postability_grade_prompt = ChatPromptTemplate.from_messages(
    [("system", postability_system), ("human", "News Article:\n\n{article}")]
)

news_chef = postability_grade_prompt | structured_llm_postability_grader

In [20]:
news_chef.invoke({"article": "한국의 GDP가 10% 증가할 것 같습니다."})

ArticlePostabilityGrader(off_or_ontopic='yes', mentions_gdp_value='yes', mentions_exchange_value='no', meets_100_words='no')

In [13]:
news_chef.invoke({"article": "K-POP의 인기가 날로 높아지고 있습니다."})

ArticlePostabilityGrader(off_or_ontopic='no', mentions_market_value='no', mentions_current_club='no', meets_100_words='no')

# Workflow

In [21]:
from typing import TypedDict, Literal
from langgraph.graph import END, StateGraph


class InputArticleState(TypedDict):
    article: str


class OutputFinalArticleState(TypedDict):
    final_article: str
    off_or_ontopic: str


class SharedArticleState(InputArticleState, OutputFinalArticleState):
    mentions_gdp_value: str
    mentions_exchange_value: str
    meets_100_words: str


def update_article_state(state: SharedArticleState) -> SharedArticleState:
    response = news_chef.invoke({"article": state["article"]})
    state["off_or_ontopic"] = response.off_or_ontopic
    state["mentions_gdp_value"] = response.mentions_gdp_value
    state["mentions_exchange_value"] = response.mentions_exchange_value
    state["meets_100_words"] = response.meets_100_words
    print("State after update_article_state:", state)
    return state


def gdp_researcher_node(state: SharedArticleState) -> SharedArticleState:
    response = gdp_research_agent.invoke({"article": state["article"]})
    state["article"] += f" {response['agent_output']}"
    return state


def current_exchange_researcher_node(state: SharedArticleState) -> SharedArticleState:
    response = exchange_rate_agent.invoke({"article": state["article"]})
    state["article"] += f" {response['agent_output']}"
    return state


def word_count_rewriter_node(state: SharedArticleState) -> SharedArticleState:
    response = text_writer_agent.invoke({"article": state["article"]})
    state["article"] += f" {response['agent_output']}"
    state["final_article"] = response["agent_output"]
    return state


def news_chef_decider(
    state: SharedArticleState,
) -> Literal[
    "gdp_researcher", "current_exchange_researcher", "word_count_rewriter", END
]:
    if state["off_or_ontopic"] == "no":
        return END
    if state["mentions_gdp_value"] == "no":
        return "gdp_researcher"
    elif state["mentions_exchange_value"] == "no":
        return "current_exchange_researcher"
    elif (
        state["meets_100_words"] == "no"
        and state["mentions_gdp_value"] == "yes"
        and state["mentions_exchange_value"] == "yes"
    ):
        return "word_count_rewriter"
    else:
        return END


workflow = StateGraph(
    SharedArticleState, input=InputArticleState, output=OutputFinalArticleState
)

workflow.add_node("news_chef", update_article_state)
workflow.add_node("gdp_researcher", gdp_researcher_node)
workflow.add_node("current_exchange_researcher", current_exchange_researcher_node)
workflow.add_node("word_count_rewriter", word_count_rewriter_node)

workflow.set_entry_point("news_chef")

workflow.add_conditional_edges(
    "news_chef",
    news_chef_decider,
    {
        "gdp_researcher": "gdp_researcher",
        "current_exchange_researcher": "current_exchange_researcher",
        "word_count_rewriter": "word_count_rewriter",
        END: END,
    },
)

workflow.add_edge("gdp_researcher", "news_chef")
workflow.add_edge("current_exchange_researcher", "news_chef")
workflow.add_edge("word_count_rewriter", "news_chef")

app = workflow.compile()

In [22]:
app.invoke({"article": "K-POP의 인기가 날로 높아지고 있습니다."})

State after update_article_state: {'article': 'K-POP의 인기가 날로 높아지고 있습니다.', 'off_or_ontopic': 'no', 'mentions_gdp_value': 'no', 'mentions_exchange_value': 'no', 'meets_100_words': 'no'}


{'off_or_ontopic': 'no'}

In [24]:
app.invoke({"article": "한국의 GDP는 환율이 10% 올라가면 어떤 영향을 받을까?"})

State after update_article_state: {'article': '한국의 GDP는 환율이 10% 올라가면 어떤 영향을 받을까?', 'off_or_ontopic': 'yes', 'mentions_gdp_value': 'yes', 'mentions_exchange_value': 'yes', 'meets_100_words': 'no'}
State after update_article_state: {'article': '한국의 GDP는 환율이 10% 올라가면 어떤 영향을 받을까? 한국의 국내총생산(GDP)은 환율 변동에 따라 여러 측면에서 영향을 받을 수 있습니다. 예를 들어, 환율이 10% 상승하게 되면 한국 원화의 가치가 달러와 같은 외화에 비해 하락하게 됩니다. 이 경우 수출품의 가격 경쟁력이 상승하여 한국 기업들이 해외 시장에서 더 많은 수출을 할 가능성이 커집니다. 결과적으로, 수출 증가로 인한 기업 매출 상승은 GDP 성장에 긍정적인 영향을 미칠 수 있습니다. 그러나 반대로, 수입품의 가격이 상승하게 되어 소비자 물가가 오를 수 있으며, 이는 가계의 구매력 감소로 이어질 수 있습니다. 따라서 환율 상승은 수출과 수입 간의 상호작용을 통해 한국 경제에 복합적인 영향을 미치게 되어, GDP의 전반적인 변동성이 커질 수 있습니다. 이러한 원인과 결과를 고려할 때, 환율 변화는 한국 경제의 성장 전망에 중요한 요소로 작용할 수 있음을 알 수 있습니다.', 'final_article': '한국의 국내총생산(GDP)은 환율 변동에 따라 여러 측면에서 영향을 받을 수 있습니다. 예를 들어, 환율이 10% 상승하게 되면 한국 원화의 가치가 달러와 같은 외화에 비해 하락하게 됩니다. 이 경우 수출품의 가격 경쟁력이 상승하여 한국 기업들이 해외 시장에서 더 많은 수출을 할 가능성이 커집니다. 결과적으로, 수출 증가로 인한 기업 매출 상승은 GDP 성장에 긍정적인 영향을 미칠 수 있습니다. 그러나 반대로, 수입품의 가격이 상승하게 되어 소비자 물가가 오를 수

{'final_article': '한국의 국내총생산(GDP)은 환율 변동에 따라 여러 측면에서 영향을 받을 수 있습니다. 예를 들어, 환율이 10% 상승하게 되면 한국 원화의 가치가 달러와 같은 외화에 비해 하락하게 됩니다. 이 경우 수출품의 가격 경쟁력이 상승하여 한국 기업들이 해외 시장에서 더 많은 수출을 할 가능성이 커집니다. 결과적으로, 수출 증가로 인한 기업 매출 상승은 GDP 성장에 긍정적인 영향을 미칠 수 있습니다. 그러나 반대로, 수입품의 가격이 상승하게 되어 소비자 물가가 오를 수 있으며, 이는 가계의 구매력 감소로 이어질 수 있습니다. 따라서 환율 상승은 수출과 수입 간의 상호작용을 통해 한국 경제에 복합적인 영향을 미치게 되어, GDP의 전반적인 변동성이 커질 수 있습니다. 이러한 원인과 결과를 고려할 때, 환율 변화는 한국 경제의 성장 전망에 중요한 요소로 작용할 수 있음을 알 수 있습니다.',
 'off_or_ontopic': 'yes'}